In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import threading
#from difflib import SequenceMatcher
import re

In [23]:
import warnings
warnings.filterwarnings("ignore")

driver1 = webdriver.Chrome(ChromeDriverManager().install())
# driver2 = webdriver.Chrome(ChromeDriverManager().install())
# driver3 = webdriver.Chrome(ChromeDriverManager().install())

author_profiles=[] 

def profiling1(link1,keywords):
    keywords = re.split(',|\s+|_', keywords)
    keywords = set(keywords)
    authorScore = 0
    
    driver1.get(link1)
    driver1.implicitly_wait(5)
    
    try:
        l = driver1.find_element("link text", "Show More")
        driver1.execute_script('arguments[0].click()', l)
    except:
        pass
    
    name = (driver1.find_elements(By.XPATH,"//h1[contains(@class,'hide-mobile')]"))
    publication_topics_list = driver1.find_elements(By.XPATH,"//div[contains(@class,'research-areas')]")
    publication_count = (driver1.find_elements(By.XPATH,"//div[contains(@class,'publications col-6 text-base-md-lh')]"))
    citations = (driver1.find_elements(By.XPATH,"//div[contains(@class,'citations col-6')]"))
    startYear = (driver1.find_elements(By.XPATH,"//span[contains(@class,'start-year col-6')]"))
    endYear = (driver1.find_elements(By.XPATH,"//span[contains(@class,'end-year col-6')]"))
    
    timePeriod = int((int(endYear[1].text))-(int(startYear[1].text)))+1
    
    citationsPerPaper = int(int((int(citations[1].text.split("\n")[1]))) / (int((publication_count[0].text).split("\n")[1])))
    publicationsPerYear = int((int((publication_count[0].text).split("\n")[1])) / (timePeriod))
    
    name = name[0].text
    publication_topics_list = (((publication_topics_list[0].text).replace("Publication Topics","")).strip())
    publication_topics_list = re.split(',|\s+', publication_topics_list)
    publication_topics_list = set(publication_topics_list)
    print(name)
    print(publication_topics_list)
    print(citationsPerPaper)
    print(publicationsPerYear)
    print(keywords)
    
    try:
        l = driver1.find_element("link text", "Show More")
        driver1.execute_script('arguments[0].click()', biography)
        biography = ((driver1.find_elements(By.XPATH,"//div[contains(@class,'biography')]"))[0]).text 
        print(biography)
    except:
        pass
    
    return author_profiles;


def authorProfiling(list):
    
    number_of_authors = len(list)-1
    keywords = list[0]
    authors=[]
    for itr in range(1,number_of_authors+1,1):
        authors.append(list[itr])
    if(number_of_authors==1):
        profiling1(str(authors[0]),keywords,)
    
    driver1.close()
#     driver2.close()
#     driver3.close()
    
#     elif(number_of_authors==2):
#         author1 = str(authors[0])
#         author2 = str(authors[1])
        
#         t1 = threading.Thread(target=profiling1,args=(author1,keywords,))
#         t2 = threading.Thread(target=profiling2,args=(author2,keywords,))
       
#         t1.start()
#         t2.start()
        
#         t1.join()
#         t2.join()
#     else:
#         author1 = str(authors[0])
#         author2 = str(authors[1])
#         author3 = str(authors[2])
        
#         t1 = threading.Thread(target=profiling1,args=(author1,keywords,))
#         t2 = threading.Thread(target=profiling2,args=(author2,keywords,))
#         t3 = threading.Thread(target=profiling3,args=(author3,keywords,))
       
#         t1.start()
#         t2.start()
#         t2.start()
        
#         t1.join()
#         t2.join()
#         t3.join()


In [24]:
authorProfiling(["image theory analysis and image","https://ieeexplore.ieee.org/author/37085562899"])

Sunil Dutt
{'chips', 'Show', 'electronics', 'field', 'gate', 'theory', 'arithmetic', 'detection', 'random', 'arrays', 'CMOS', 'digital', 'silicon', 'Less', 'number', 'low-power', 'tuning', 'semiconductors', 'processing', 'integrated', 'adders', 'approximation', 'semiconductor', 'correction', 'design', 'cryptography', 'signal', 'carry', 'circuits', 'elemental', 'yield', 'circuit', 'industry', 'generation', 'multiplying', 'simulation', 'logic', 'error', 'programmable'}
6
1
{'image', 'analysis', 'theory', 'and'}
